In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import gmres, spilu, LinearOperator
from scipy.sparse import csr_matrix
from scipy.integrate import dblquad
from scipy.special import erf
import imageio
import os
import warnings
warnings.filterwarnings('ignore')

### Numeric solution

In [2]:
def gmres_solve(A, b, N, use_prec=True):
    if use_prec:
        ILU = spilu(A, fill_factor=1.0, drop_tol=1e-3)
        prec = LinearOperator(((N-1)**2, (N-1)**2), matvec=ILU.solve)
        return gmres(A, b, M=prec, tol=1e-6)[0]
    else:
        return gmres(A, b, tol=1e-6)[0]

In [3]:
integrals = dict()
integrals['phi_phi']     = np.array([[1/36, 1/9, 1/36],
                                     [1/9, 4/9, 1/9],
                                     [1/36, 1/9, 1/36]]).T
integrals['phi_dxphi']   = np.array([[1/12, 0, -1/12],
                                     [1/3, 0, -1/3],
                                     [1/12, 0, -1/12]]).T
integrals['dxphi_phi']   = np.array([[-1/12, 0, 1/12], 
                                     [-1/3, 0, 1/3], 
                                     [-1/12, 0, 1/12]]).T
integrals['dxphi_dxphi'] = np.array([[-1/6, 1/3, -1/6],
                                     [-2/3, 4/3, -2/3],
                                     [-1/6, 1/3, -1/6]]).T
integrals['dyphi_dyphi'] = np.array([[-1/6, -2/3, -1/6],
                                     [1/3, 4/3, 1/3],
                                     [-1/6, -2/3, -1/6]]).T

In [4]:
c_0 = 1
a = 10
d_x, d_y = 1e-4, 1e-1

#square [0:200] x [-100:100]
N = 200
h = 200.0 / N

P = h/d_x # Peclet number

supg = True
if supg and P >= 1:
    delta_e = (h - d_x) / 2
else:
    delta_e = 0

g_D = lambda x, y: c_0 if x == 0 and abs(y) < 10 else 0

T = 50
dt = h

In [5]:
u_prev = np.zeros((N+1)**2)

for j in range(0, N):
    u_prev[(N+1)*j] = g_D(0, -100+h*j)
solutions = [u_prev]

filenames = []

for t in np.arange(0, T, dt):
    row_start = [0]
    col_id = []
    data = []
    b = []

    for j in range(1, N):
        for i in range(1, N):
            b_coef = 0
            ncoefs = 0
            for it_k in [-1, 0, 1]:
                for it_l in [-1, 0, 1]:
                    int_k, int_l = it_k + 1, it_l + 1
                    k, l = i + it_k, j + it_l

                    C_1 = h**2 * integrals['phi_phi'][int_k, int_l] + \
                         delta_e * h * integrals['phi_dxphi'][int_k, int_l]
                    C_2 = dt * (h * integrals['dxphi_phi'][int_k, int_l] * h + \
                                delta_e * integrals['dxphi_dxphi'][int_k, int_l])
                    C_3 = dt * (d_x * integrals['dxphi_dxphi'][int_k, int_l] + \
                                d_y * integrals['dyphi_dyphi'][int_k, int_l])
                    
                    if 0 < k < N and 0 < l < N:
                        col_id.append(k-1 + (N-1)*(l-1))
                        data.append(C_1 + C_2 + C_3)
                        b_coef += C_1 * u_prev[k + (N+1)*l]
                        ncoefs += 1
                    else:
                        b_coef += -(C_2 + C_3) * u_prev[k + (N+1)*l]
            
            b.append(b_coef)   
            row_start.append(row_start[-1] + ncoefs)
                    
    A = csr_matrix((data, col_id, row_start))

    u_sol = gmres_solve(A, b, N).reshape((N-1, N-1), order='F')
    u_prev = u_prev.reshape((N+1, N+1), order='F')
    u_prev[1:N, 1:N] = u_sol.copy()
    solutions.append(u_prev)
    
    filename = f'./gif/{t+1}.png'
    filenames.append(filename)

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.set_xticks([0, 20, 40, 60, 80, 100])
    ax.set_yticks([0, 10, 20, 30, 40])
    ax.set_yticklabels([-20, -10, 0, 10, 20])
    ax.set_title(fr'$u_h(x, y, t={t+1:.1f}$)')
    ax.imshow(u_prev[0:(N // 2)+1, (4 * N // 10)-1:(6 * N // 10)+1].T, origin='lower')
    plt.savefig(filename, dpi=200)
    plt.close()
    u_prev = u_prev.reshape(((N+1)**2), order='F')

In [6]:
images = []
for filename in filenames:
   images.append(imageio.imread(filename))
imageio.mimsave('./gif/advection.gif', images)

### Analytic solution

In [7]:
def eval_func(x, y, t, n=10**3):
    def f(x, y, tau):
        return tau**(-3/2) * (erf((a+y)/np.sqrt(4*d_y*tau)) + erf((a-y)/np.sqrt(4*d_y*tau))) * np.exp(-(x-tau)**2/(4*d_x*tau))
    
    def theta(j):
        return np.cos(np.pi*(2*j-1) / (2*n))
        
    vec_1 = np.fromfunction(lambda j: np.sqrt(1 - theta(j)**2), (n+1,))[1:]
    vec_2 = np.fromfunction(lambda j: f(x, y, t * (theta(j) + 1) / 2), (n+1,))[1:]

    return x*c_0 / np.sqrt(16 * np.pi * d_x) * np.pi*t / (2*n) * np.dot(vec_1, vec_2)

In [8]:
filenames = []
true_solutions = []
for t in np.arange(1, T, 5):
    u_true = np.fromfunction(lambda i, j: np.vectorize(eval_func)(i*h, j*h - 100, t), (N+1, N+1))
    for j in range(0, N):
        u_true[0][j] = g_D(0, -100+h*j)
    true_solutions.append(u_true)

    filename = f'./gif_true/{t+1}.png'
    filenames.append(filename)

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.set_xticks([0, 20, 40, 60, 80, 100])
    ax.set_yticks([0, 10, 20, 30, 40])
    ax.set_yticklabels([-20, -10, 0, 10, 20])
    ax.set_title(fr'$u(x, y, t={t+1:.1f}$)')
    ax.imshow(u_true[0:(N // 2)+1, (4 * N // 10)-1:(6 * N // 10)+1].T, origin='lower')
    plt.savefig(filename, dpi=200)
    plt.close()

In [9]:
images = []
for filename in filenames:
   images.append(imageio.imread(filename))
imageio.mimsave('./gif_true/advection.gif', images)

### Error evaluation

In [10]:
t = 50
u_true = np.fromfunction(lambda i, j: np.vectorize(eval_func)(i*h, j*h - 100, t), (N+1, N+1))
for j in range(0, N):
    u_true[0][j] = g_D(0, -100+h*j)

print(f'C-norm error: {np.abs(solutions[t] - u_true).max()}')

C-norm error: 0.4482543242671212
